In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

In [13]:
import os
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout,Dense
from keras.optimizers import Adam

In [22]:
CLASS_NUM = 10
IMAGE_SIZE_W = 28
IMAGE_SIZE_H = 28
FLATTEN_SIZE = IMAGE_SIZE_W*IMAGE_SIZE_H

In [42]:
train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")

from sklearn.model_selection import train_test_split
y = "label"
X_train, X_test, y_train, y_test = train_test_split(train_data.drop(y, axis=1), 
                                                        train_data[y],
                                                        stratify=train_data[y],
                                                        random_state=0,
                                                        train_size=0.6)

X_train /= 255
X_test /= 255
test_data /= 255

X_train = X_train.as_matrix().reshape(-1, IMAGE_SIZE_H, IMAGE_SIZE_W, 1)
X_test = X_test.as_matrix().reshape(-1, IMAGE_SIZE_H, IMAGE_SIZE_W, 1)
test_data = test_data.as_matrix().reshape(-1, IMAGE_SIZE_H, IMAGE_SIZE_W, 1)

print("X_train:{}\ny_train:{}".format(X_train.shape, y_train.shape))
print("X_test :{}\ny_test :{}".format(X_test.shape, y_test.shape))
print("test_data :{}".format(test_data.shape))

C:\Users\tomiyou\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


X_train:(25200, 28, 28, 1)
y_train:(25200,)
X_test :(16800, 28, 28, 1)
y_test :(16800,)
test_data :(28000, 28, 28, 1)


In [43]:
y_train = np_utils.to_categorical(y_train, CLASS_NUM)
y_test = np_utils.to_categorical(y_test, CLASS_NUM)

In [52]:
EPOCH_SIZE = 8
BATCH_SIZE = 128

### LeNet

In [53]:
input_shape = (IMAGE_SIZE_H, IMAGE_SIZE_W, 1)

lenet_model = Sequential()
lenet_model.add(Conv2D(20, kernel_size=5, padding="same", input_shape=input_shape, activation="relu"))
lenet_model.add(MaxPooling2D(pool_size=(2, 2)))
lenet_model.add(Conv2D(50, kernel_size=5, padding="same", activation="relu"))
lenet_model.add(MaxPooling2D(pool_size=(2, 2)))
lenet_model.add(Flatten())
lenet_model.add(Dense(120, activation="relu"))
lenet_model.add(Dense(64, activation="relu"))
lenet_model.add(Dense(CLASS_NUM))
lenet_model.add(Activation("softmax"))

In [54]:
lenet_model.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy"])

lenet_history = lenet_model.fit(X_train, y_train, verbose=1,
             epochs=EPOCH_SIZE, batch_size=BATCH_SIZE, validation_split=0.3)

Train on 17640 samples, validate on 7560 samples
Epoch 1/8
17640/17640 [==============================] - 45s 3ms/step - loss: 0.4684 - acc: 0.8572 - val_loss: 0.1411 - val_acc: 0.9582
Epoch 2/8
17640/17640 [==============================] - 44s 2ms/step - loss: 0.1055 - acc: 0.9681 - val_loss: 0.0746 - val_acc: 0.9763
Epoch 3/8
17640/17640 [==============================] - 42s 2ms/step - loss: 0.0625 - acc: 0.9804 - val_loss: 0.0724 - val_acc: 0.9747
Epoch 4/8
17640/17640 [==============================] - 42s 2ms/step - loss: 0.0454 - acc: 0.9859 - val_loss: 0.0443 - val_acc: 0.9858
Epoch 5/8
17640/17640 [==============================] - 42s 2ms/step - loss: 0.0352 - acc: 0.9891 - val_loss: 0.0507 - val_acc: 0.9840
Epoch 6/8
17640/17640 [==============================] - 41s 2ms/step - loss: 0.0285 - acc: 0.9906 - val_loss: 0.0544 - val_acc: 0.9836
Epoch 7/8
17640/17640 [==============================] - 42s 2ms/step - loss: 0.0204 - acc: 0.9939 - val_loss: 0.0516 - val_acc: 0.9837

In [55]:
lenet_model.evaluate(X_test, y_test, verbose=1)

16800/16800 [==============================] - 15s 880us/step


[0.057229259130919155, 0.9826785714285714]

In [56]:
pred_y = lenet_model.predict(test_data)
results = np.argmax(pred_y,axis = 1)
results = pd.Series(results, name="Label")
submission = pd.concat([pd.Series(range(1,28001), name = "ImageId"),results], axis = 1)
submission.to_csv("digit_recognizer_cnn.csv",index=False) # 0.9850

### Data Augmentation

In [65]:
EPOCH_SIZE = 2
BATCH_SIZE = 128

In [66]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

In [67]:
lenet_model.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy"])
history_datagen = lenet_model.fit_generator(datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                              epochs = EPOCH_SIZE, validation_data = (X_test, y_test),
                              verbose = 1, steps_per_epoch=X_train.shape[0] // BATCH_SIZE)

Epoch 1/2
196/196 [==============================] - 69s 353ms/step - loss: 0.0912 - acc: 0.9729 - val_loss: 0.0436 - val_acc: 0.9864
Epoch 2/2
196/196 [==============================] - 68s 347ms/step - loss: 0.0660 - acc: 0.9785 - val_loss: 0.0488 - val_acc: 0.9849


In [68]:
pred_y = lenet_model.predict(test_data)
results = np.argmax(pred_y,axis = 1)
results = pd.Series(results, name="Label")
submission = pd.concat([pd.Series(range(1,28001), name = "ImageId"),results], axis = 1)
submission.to_csv("digit_recognizer_cnn_da.csv",index=False) # 0.9850